<a href="https://colab.research.google.com/github/RayNCode/code_collab/blob/main/BestCominations_LogRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook. I run models, based on the datasets I retrieved from Main_1.

In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [ ]:
X = pd.read_csv('/content/X_Data.csv', low_memory=False)
df = pd.read_csv('/content/target_data.csv')

In [ ]:
y = df['target'].copy()
y = np.where(y == 'B', 1, 0)
y

array([1, 1, 0, ..., 1, 0, 1])

In [ ]:
X.dtypes

insee_code            object
is_student              bool
OCCUPATION_42         object
ACTIVITY_TYPE         object
AGE_2019               int64
household             object
Highest_degree        object
sex                   object
employer_category     object
job_category          object
EMPLOYEE_COUNT        object
Terms_of_emp          object
Eco_sect              object
work_description      object
Job_dep               object
Working_hours        float64
WORK_CONDITION        object
EMOLUMENT            float64
N3                    object
N2                    object
N1                    object
town_type             object
dep                   object
Emp_contract          object
Club                  object
Categorie             object
REG                    int64
dtype: object

In [ ]:
X['REG'] = X['REG'].astype('object')
X.dtypes

insee_code            object
is_student              bool
OCCUPATION_42         object
ACTIVITY_TYPE         object
AGE_2019               int64
household             object
Highest_degree        object
sex                   object
employer_category     object
job_category          object
EMPLOYEE_COUNT        object
Terms_of_emp          object
Eco_sect              object
work_description      object
Job_dep               object
Working_hours        float64
WORK_CONDITION        object
EMOLUMENT            float64
N3                    object
N2                    object
N1                    object
town_type             object
dep                   object
Emp_contract          object
Club                  object
Categorie             object
REG                   object
dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn
sklearn.set_config(display="diagram")
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from itertools import combinations

In [ ]:
X = X.drop(["work_description"], axis="columns")

In [ ]:
X.columns

Index(['insee_code', 'is_student', 'OCCUPATION_42', 'ACTIVITY_TYPE',
       'AGE_2019', 'household', 'Highest_degree', 'sex', 'employer_category',
       'job_category', 'EMPLOYEE_COUNT', 'Terms_of_emp', 'Eco_sect', 'Job_dep',
       'Working_hours', 'WORK_CONDITION', 'EMOLUMENT', 'N3', 'N2', 'N1',
       'town_type', 'dep', 'Emp_contract', 'Club', 'Categorie', 'REG'],
      dtype='object')

In [ ]:
preprocessing = make_column_transformer (
    (OneHotEncoder(handle_unknown='ignore'), ['insee_code',"is_student", "OCCUPATION_42", "ACTIVITY_TYPE", "household", "sex", "employer_category", "job_category", "Terms_of_emp",
                       "Eco_sect", "Job_dep", "WORK_CONDITION", "N3", "N2", "N1", "town_type", "dep", "Emp_contract", "Club", "Categorie", 'REG']),
    (OrdinalEncoder(), ["Highest_degree", "EMPLOYEE_COUNT"]),
    remainder='passthrough')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [ ]:
# Modèle de régression logistique
logreg = LogisticRegression(solver='lbfgs', max_iter=5000)

# Colonnes pour tester les combinaisons
columns_to_combine = ['insee_code', 'is_student', 'OCCUPATION_42', 'ACTIVITY_TYPE', 'AGE_2019', 'household',
                      'Highest_degree', 'sex', 'employer_category', 'job_category', 'EMPLOYEE_COUNT', 'Terms_of_emp',
                      'Eco_sect', 'Job_dep', 'Working_hours', 'WORK_CONDITION', 'EMOLUMENT', 'N3', 'N2', 'N1',
                      'town_type', 'dep', 'Emp_contract', 'Club', 'Categorie', 'REG']

# Stocker le meilleur score et la meilleure combinaison
best_score = 0
best_combination = []

In [ ]:
def create_preprocessing_pipeline(columns):
    """
    Crée un pipeline de prétraitement adapté aux colonnes spécifiées.
    """
    # Définir les colonnes pour chaque type de prétraitement
    one_hot_cols = [col for col in ['insee_code', 'is_student', 'OCCUPATION_42', 'ACTIVITY_TYPE', 'household',
                                    'sex', 'employer_category', 'job_category', 'Terms_of_emp', 'Eco_sect',
                                    'Job_dep', 'WORK_CONDITION', 'N3', 'N2', 'N1', 'town_type', 'dep',
                                    'Emp_contract', 'Club', 'Categorie', 'REG'] if col in columns]
    ordinal_cols = [col for col in ['Highest_degree', 'EMPLOYEE_COUNT'] if col in columns]

    # Créer le transformateur de colonnes
    transformers = []
    if one_hot_cols:
        transformers.append((OneHotEncoder(handle_unknown='ignore'), one_hot_cols))
    if ordinal_cols:
        transformers.append((OrdinalEncoder(), ordinal_cols))

    # S'il n'y a aucun transformateur, retourner 'passthrough' pour traiter toutes les colonnes telles quelles
    if not transformers:
        return 'passthrough'

    return make_column_transformer(*transformers, remainder='passthrough')

# Réinitialiser les meilleures valeurs
best_score = 0
best_combination = []

# Tester toutes les combinaisons possibles à partir de la première colonne
for r in range(1, len(columns_to_combine) + 1):
    for combination in combinations(columns_to_combine, r):
        # Sélectionner les colonnes pour la combinaison actuelle
        X_train_comb = X_train[list(combination)]
        X_test_comb = X_test[list(combination)]

        # Créer le prétraitement adapté à la combinaison actuelle
        preprocessing_comb = create_preprocessing_pipeline(combination)

        # Pipeline avec prétraitement adapté et régression logistique
        pipe_logreg = make_pipeline(preprocessing_comb, logreg)

        # Calculer le score moyen avec validation croisée
        try:
            score = cross_val_score(pipe_logreg, X_train_comb, y_train, cv=3, scoring="accuracy", error_score='raise').mean()

        except ValueError:
            # Gérer les erreurs dues à des combinaisons de colonnes inappropriées
            continue

        # Mettre à jour le meilleur score et la combinaison si nécessaire
        if score > best_score:
            best_score = score
            best_combination = combination
            print(f"The best score is {best_score}, and combination: {combination}")
# Retourner le meilleur score et la meilleure combinaison de colonnes
best_score, best_combination

The best score is 0.6259439142473996, and combination: ('insee_code',)
The best score is 0.7191578770809345, and combination: ('OCCUPATION_42',)
The best score is 0.7538630203880038, and combination: ('insee_code', 'OCCUPATION_42')
The best score is 0.7562884823475972, and combination: ('OCCUPATION_42', 'dep')
The best score is 0.7591637886315074, and combination: ('insee_code', 'OCCUPATION_42', 'ACTIVITY_TYPE')
The best score is 0.7772165578271127, and combination: ('insee_code', 'OCCUPATION_42', 'household')
The best score is 0.7839925917428588, and combination: ('OCCUPATION_42', 'household', 'dep')
The best score is 0.7859928855386475, and combination: ('insee_code', 'OCCUPATION_42', 'ACTIVITY_TYPE', 'household')
The best score is 0.7875931149487313, and combination: ('insee_code', 'OCCUPATION_42', 'household', 'employer_category')
The best score is 0.7878181468285602, and combination: ('insee_code', 'OCCUPATION_42', 'household', 'Eco_sect')
The best score is 0.791293711923139, and 

KeyboardInterrupt: 

In [ ]:
logreg = LogisticRegression(solver='lbfgs', max_iter=5000)
pipe_logreg = make_pipeline(preprocessing, logreg)
pipe_logreg

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['insee_code', 'is_student',
                                                   'OCCUPATION_42',
                                                   'ACTIVITY_TYPE', 'household',
                                                   'sex', 'employer_category',
                                                   'job_category',
                                                   'Terms_of_emp', 'Eco_sect',
                                                   'Job_dep', 'WORK_CONDITION',
                                                   'N3', 'N2', 'N1',
                                                   'town_type', 'dep',
                                                   'Emp_contract', 'Club',
                                                   'Categorie', 'REG']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Highest_degree',
                                                   'EMPLOYEE_COUNT'])])),
                ('logisticregression', LogisticRegression(max_iter=5000))])

In [ ]:
cross_val_score(pipe_logreg, X_train, y_train, cv=3, scoring="accuracy", error_score='raise').mean()

0.7787402436890604

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
pipe_forest = make_pipeline(preprocessing, forest_clf)
cross_val_score(pipe_forest, X_train, y_train, cv=5, scoring="accuracy", n_jobs=-1).mean()

0.8570036589657478

In [ ]:
pipe_forest.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['insee_code', 'is_student',
                                                   'OCCUPATION_42',
                                                   'ACTIVITY_TYPE', 'household',
                                                   'sex', 'employer_category',
                                                   'job_category',
                                                   'Terms_of_emp', 'Eco_sect',
                                                   'Job_dep', 'WORK_CONDITION',
                                                   'N3', 'N2', 'N1',
                                                   'town_type', 'dep',
                                                   'Emp_contract', 'Club',
                                                   'Categorie', 'REG']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Highest_degree',
                                                   'EMPLOYEE_COUNT'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

In [ ]:
# Accédez au modèle RandomForestClassifier à l'intérieur du pipeline
forest_model = pipe_forest.named_steps['randomforestclassifier']

# Obtenez les importances des caractéristiques du modèle
feature_importances = forest_model.feature_importances_

In [ ]:
# Accédez à l'objet ColumnTransformer dans votre pipeline.
preprocessor = pipe_forest.named_steps['columntransformer']

# Utilisez la méthode get_feature_names_out() pour obtenir les noms des features.
feature_names = preprocessor.get_feature_names_out()

# Reliez ces noms aux importances des caractéristiques pour les interpréter.
feature_importances = pipe_forest.named_steps['randomforestclassifier'].feature_importances_

feature_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# Trier le DataFrame par importance pour voir les caractéristiques les plus importantes.
feature_importances_df.sort_values(by='Importance', ascending=False)

,Feature,Importance
12847,remainder__AGE_2019,0.045877
12845,ordinalencoder__Highest_degree,0.028945
12848,remainder__Working_hours,0.023809
12849,remainder__EMOLUMENT,0.020648
12620,onehotencoder__town_type_Commune simple,0.016035
...,...,...
6758,onehotencoder__insee_code_57345,0.000000
11636,onehotencoder__insee_code_91156,0.000000
2368,onehotencoder__insee_code_24481,0.000000
7763,onehotencoder__insee_code_62337,0.000000
